# Free Flame Example

This example uses `ctwrap` for one-dimensional adiabatic premixed flame simulations defined by the simulation module `freeflame` (`ctwrap.modules.freeflame`).

## Function/Class Definitions

In [ ]:
import cantera as ct
import ctwrap as cw
import json

## Simulations

### Create Simulation Object

Uses a module in the `modules` folder. Modules do not have to depend on `ctwrap`.

In [ ]:
sim = cw.Simulation.from_module(cw.modules.freeflame)
sim

### Create Simulation Handler

In [ ]:
fname = 'freeflame.yaml'
s = cw.SimulationHandler.from_yaml(fname, strategy='matrix', verbosity=1)
s

### Run Simulation Tasks in Parallel

In [ ]:
%%time
s.run_parallel(sim, verbosity=0)

## Create Graphical Output

In [ ]:
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
col = mpl.rcParams['axes.prop_cycle'].by_key()['color']
n_col = len(col)

### Load Data

In [ ]:
data = h5py.File(s.output_name, 'r')
data.keys()

In [ ]:
data.attrs.keys()

In [ ]:
var = json.loads(data.attrs['strategy'])
var

In [ ]:
default = json.loads(data.attrs['defaults'])
default

In [ ]:
up = default.get('upstream')
up

In [ ]:
cases = json.loads(data.attrs['cases'])
cases

In [ ]:
phi = [t['upstream.phi'] for t in cases.values() 
       if t['model.transport'] == 'mix']
phi

In [ ]:
mix = {k: dict(data[k]['flame']) for k, v in cases.items() 
       if v['model.transport'] == 'mix'}
mlt = {k: dict(data[k]['flame']) for k, v in cases.items() 
       if v['model.transport'] == 'multi'}
sor = {k: dict(data[k]['flame']) for k, v in cases.items() 
       if v['model.transport'] == 'soret'}

In [ ]:
mix.keys()

### Plot

Plot flame speeds and temperature profiles

In [ ]:
# create figure
fig, ax = plt.subplots(1)
fig.set_size_inches(8.,6.)

# plot results
u_mix = [ mix[t]['velocity'][0] for t in mix]
u_mlt = [ mlt[t]['velocity'][0] for t in mlt]
u_sor = [ sor[t]['velocity'][0] for t in sor]

ax.plot(phi, u_mix, marker='o', 
            linestyle='none', label='mixture-averaged') 
ax.plot(phi, u_mlt, marker='s', 
            markerfacecolor='none',
            linestyle='none', label='multi-component') 
ax.plot(phi, u_sor, marker='x', 
            markerfacecolor='none',
            linestyle='none', label='soret enabled') 

# add title/axis labels
ax.set_xlabel('Equivalence ratio (-)')
ax.set_ylabel('Flame speed (m/s)')

ax.set_title('Free premixed flame simulation for {} with {}'.format(up['fuel'],up['oxidizer']))

# add legend
leg = ax.legend(loc='lower right',#prop={'size':11},
                title='Transport Model') # % mech)
_ = leg.draw_frame(False) 